In [5]:
#pip install google_play_scraper
%pip install pandas

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.9 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.2/11.3 MB 6.7 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/11.3 MB 8.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/11.3 MB 11.4 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/11.3 MB 10.6 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.3 MB 10.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.3 MB 9.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/11.3 MB 9.1 MB/s eta 0:00:01
   ----------- --------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from google_play_scraper import Sort, reviews
import pandas as pd
import time, random

In [ ]:
# esto es solo para una aplicacion y los primeros 200 comentarios

#app_id = "bo.com.yolopago"

# Extraer reseñas (puedes ajustar 'count' según lo que necesites)
#result, _ = reviews(
#    app_id,
#    lang='es',
#    country='bo',
#    sort=Sort.NEWEST, # puedes usar: Sort.NEWEST, Sort.HELPFUL, Sort.RATING
#    count=200         # número de reseñas a descargar en una corrida
#)

# Convertir a DataFrame
#df = pd.DataFrame(result)

# Guardar en CSV
#df.to_csv("comentarios_app.csv", index=False, encoding="utf-8-sig")

#print("Se guardaron", len(df), "comentarios en 'comentarios_app.csv'")


In [ ]:
#https://play.google.com/store/apps/details?id=bo.com.yolopago&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.bancosol.altoke&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.bcp.bo.wallet&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.busa.wallet&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.walletapp.mobile&pcampaignid=web_share

In [ ]:
#print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              200 non-null    object        
 1   userName              200 non-null    object        
 2   userImage             200 non-null    object        
 3   content               200 non-null    object        
 4   score                 200 non-null    int64         
 5   thumbsUpCount         200 non-null    int64         
 6   reviewCreatedVersion  165 non-null    object        
 7   at                    200 non-null    datetime64[ns]
 8   replyContent          0 non-null      object        
 9   repliedAt             0 non-null      object        
 10  appVersion            165 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 17.3+ KB
None


In [ ]:
#df['content'].sample(15)

,content
22,muy útil!
115,"un desastre falla la billetera móvil,no puedo ..."
96,Muy buena aplicación
58,💪
99,"muy agradecida con el personal de atención, so..."
178,esa app esta a punto de dejar de ser compatibl...
79,pura burrera la nueva actualizacion solo es pa...
48,Muy buena🌟
107,interesante app
109,aurita mismo me están pidiendo q active mi ubi...


In [ ]:
all_reviews = []
token = None

app_id = ["com.bancosol.altoke", "com.bcp.bo.wallet", "bo.com.yolopago", "com.busa.wallet", "id=com.walletapp.mobile"]

for app in app_id:
    for _ in range(5):
        result, token = reviews(
            app,
            lang='es',
            country='bo',
            sort=Sort.NEWEST,
            count=200,
            continuation_token=token # esto te permite paginar
    )
    all_reviews.extend(result)

    if not token:  # cuando ya no hay más reseñas
        break

    time.sleep(random.uniform(5, 10))

df2 = pd.DataFrame(all_reviews)


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              3000 non-null   object        
 1   userName              3000 non-null   object        
 2   userImage             3000 non-null   object        
 3   content               3000 non-null   object        
 4   score                 3000 non-null   int64         
 5   thumbsUpCount         3000 non-null   int64         
 6   reviewCreatedVersion  2447 non-null   object        
 7   at                    3000 non-null   datetime64[ns]
 8   replyContent          772 non-null    object        
 9   repliedAt             772 non-null    datetime64[ns]
 10  appVersion            2447 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 257.9+ KB


In [8]:
df2['content'].sample(12)

305     ya van 3 veces que hago transferencia en tiend...
375                                                 super
1843    La verdad no se porque pide actualizar y me pu...
235     una basura me robó 6 bs y se cuelga al momento...
2051                                   Pele la aplicación
1761    Que Pasa Con Yape Esta Caido Me Estan Perjudic...
1222    Yape se cuelga 3 veces a la semana en Bolivia....
2245    Es buena la aplicación me ha servido de mucha ...
355     es burocrático te pide carnet de identidad al ...
1500    Que pasó con las recargas de paquetes? Ya no s...
2275    La aplicación funciona realmente pésimo .... C...
827     si yo igual creado mi cuenta ahora pago mis fa...
Name: content, dtype: object

In [9]:
df2.shape

(3000, 11)

In [11]:
df2.to_csv("../data/restore/comentarios_app.csv", index=False, encoding="utf-8-sig")
print("✅ Total de reseñas descargadas:", len(df2))

✅ Total de reseñas descargadas: 3000
